# Markov Chain Monte Carlo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

np.random.seed(123)

## The Bayesics 

A Monte Carlo routine is a random numerical walk through a parameter space that calculates possible solutions to or integrations of a given function, and each realisation of the routine provides a different outcome. By combining all of these solutions, an approximation of the most likely answer is yielded with error bars, and the correct answer is likely to be contained within those error bars. A Monte Carlo chain is considered to be Markov if its next "step" in parameter space is only dependent on its current position, and not the history of the entire chain.

These types of algorithms are usually implemented when scientists are faced with complex models for which other computational approaches are not appropriate. For example, Markov Chain Monte Carlo (MCMC) may be chosen when a least squares fitting algorithm has difficulty fitting a high-dimensional model with multiple probability wells in the parameter space. It takes a long time to calculate the probability a model is true for provided data at every point in your parameter space, and this will only increase proportionally to the parameter number. MCMC methods are designed to spend time sampling the most likely parameter sets. Also they do not require a full analytic description of the normalised probability density function (pdf) to begin sampling since only the ratios of the pdf at pairs of locations are calculated, making them ideal for calculating _posterior pdfs_.

In probabilistic inference, Bayes rule states that the posterior pdf $p\left(\theta|D\right)$ (pdf of parameters $\theta$ given data $D$) can be determined from the likelihood $p\left(D|\theta\right)$ (pdf of the data given the parameters) and the prior pdf $p\left(\theta\right)$ for the parameters.

<center>$p\left(\theta|D\right)=\frac{1}{Z}p\left(D|\theta\right)p\left(\theta\right)$</center>

Here, $Z$ is sometimes referred to as the "marginalised likelihood" and is incredibly difficult and computationally expensive to calculate. However, since only the ratio $p\left(\theta^{\prime}|D\right)/p\left(\theta|D\right)$ is important, $Z$ cancels out and the posterior pdf can be sampled.

## The Metropolis-Hastings Algorithm

The Metropolis-Hastings (M-H) step is the simplest MCMC algorithm and requires only two inputs: the posterior pdf to be sampled and a proposal algorithm that producess the samples, allowing the algorithm to walk around the parameter space in a fair and random way. The algorithm is as follows:

* Draw a proposal of $\theta^{\prime}$ from pdf $q\left(\theta^{\prime}|\theta_{k}\right)$ (this is usually a Gaussian distribution centred on the current position).
* Draw a random number $0<r<1$ from a uniform distribution.
* If $p\left(\theta^{\prime}\right)/p\left(\theta_{k}\right)>r$ then $\theta_{k+1}\leftarrow\theta^{\prime}$; otherwise $\theta_{k+1}\leftarrow\theta_{k}$.

This produces a random walk which is biased by the acceptance step involving the ratios of the function values. The amount of time the algorithm spends around a location $\theta$ is proportional to $p\left(\theta\right)$ and samples have to be significantly separated along the chain before they are considered independent from each other. Therefore, fair samples are only produced in the limit of arbitrary run time.

In [ ]:
mus = np.array([5., 5.])                   # Array of mean values
sigmas = np.array([[1., 0.9], [0.9, 1.]])  # Covariance matrix

def pgauss(x, y):
    """
Function to calculate the Probability Density Function of a Bivariate Gaussian Distribution.
    """
    return stats.multivariate_normal.pdf([x, y], mean=mus, cov=sigmas)

In [ ]:
def Metropolis_Hastings(p, iter=1000):
    """
Basic Metropolis-Hastings MCMC sampler. Produces a single Markov chain of samples from the parsed Probability Density Function.
    """
    x, y = 0., 0.                  # Initialise the Markov chain
    samples = np.zeros((iter, 2))  # Create an empty array in which to store the chain
    
    for i in range(iter):  # Iterate over the length of the chain
        # Calculate proposal based on a Gaussian distribution with a mean of the current position.
        x_new, y_new = np.array([x, y]) + np.random.normal(2)
        
        # Accept or reject stage using M-H rule.
        if np.random.rand() < (p(x_new, y_new) / p(x, y)):
            x, y = x_new, y_new
        samples[i] = np.array(x, y)
        
    return samples

In [ ]:
%%time
Nstep = 10000
print("Total sample number =", Nstep)

s = Metropolis_Hastings(pgauss, iter=Nstep)  # Run the M-H sampler
%timeit -n1 s

In [ ]:
# Plot the samples
fig = plt.figure(figsize=(6,6))

ax = plt.subplot(223)
ax.plot(s[:,0], s[:,1], '.k')
xlims = ax.get_xlim()
ylims = ax.get_ylim()
ax.set_xlabel('x')
ax.set_ylabel('y')

ax = fig.add_subplot(221)
ax.hist(s[:,0], histtype='step', color='k', density=True)
ax.set_xlim(xlims)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.add_subplot(224)
ax.hist(s[:,1], histtype='step', color='k', density=True, orientation='horizontal')
ax.set_ylim(ylims)
ax.set_xticks([])
ax.set_yticks([])

fig.tight_layout(h_pad=0., w_pad=0.)
plt.show()

## The Rosenbrock Density

The Rosenbrock density (depicted in the figure below) is an example of a highly anisotropic distribution and is given by the following pdf (Goodman & Weare, 2010).

<center>$\pi\left(x_{1},x_{2}\right)\propto\exp\left(-\frac{100\left(x_{2}-x_{1}^{2}\right)^{2}+\left(1-x_{1}\right)^{2}}{20}\right)$</center>

![Rosenbrock Density](./assets/GoodmanWeare_Rosenbrock.png)

This kind of distribution is an ideal test case for the efficiency of MCMC algorithms since its degeneracy issues make it difficult to sample. If we imagine beginning a M-H sampler in the infinitesimal probability space surrounding the Rosenbrock density, the sampler will gradually step around the space towards the density, using Gaussian proposal steps, without many issues. The problems arise when the M-H sampler reaches the ridge of the probability density and need to walk along it until it reaches the peak at (0,0). Now, any small deviation from its current position will result in the sampler falling back into infinitesimal probability space. Therefore, the algorithm becomes stuck for a period of time whilst searching for a proposal which will be accepted, wasting valuable computing time. What is required is a sampler that can investigate likely probabilities across the whole pdf without becoming stuck.

<div class="alert alert-block alert-info">
Generally, this problem is solved by defining a linear operator that transforms the parameter space into one where Gaussian steps can easily sample the distribution, but this becomes difficult when working with a high-dimensional model.
</div>

## Affine Invariant Ensemble Sampler

The ability to navigate awkwardly transformed distributions without becoming stuck is known as *affine invariance*. This means that the performance of the method is independent of the aspect ratio in highly anisotropic distributions. A linear affine transformation is the mapping of $\mathbb{R}^{n}$ to the form $y=Ax+b$. If $X$ has a probability of density of $\pi\left(x\right)$, then $Y=Ax+b$ has the density $\pi_{A,b}\left(y\right)=\pi_{A,b}\left(Ax+b\right)\propto\pi\left(x\right)$. For example, given the density

<center>$\pi\left(x\right)\propto\exp\Big[\frac{-\left(x_{1}-x_{2}\right)^{2}}{2\epsilon}-\frac{\left(x_{1}+x_{2}\right)^{2}}{2}\Big]$</center>

perturbations of order $\sqrt\epsilon$ and $1$ can be applied to each term respectively to produce that transformations

<center>$y_{1}=\frac{x_{1}-x_{2}}{\sqrt\epsilon}$, $y_{2}=x_{1}+x_{2}$.</center>
    
Hence deriving a much simpler distribution to sample in the form of:

<center>$\pi_{A}\left(y\right)\propto\exp\Big[-\frac{\left(y_{1}^{2}+y_{2}^{2}\right)}{2}\Big]$.</center>


The algorithm used to perform an affine invariant transformation and draw proposals from it is known as the "stretch move", depicted in the figure below (Goodman & Weare, 2010).

* To move a walker $X_{k}\left(t\right)$, another walker $X_{j}$ is randomly chosen from the rest of the ensemble (i.e. $j\ne k$).
* A new position is chosen from a random linear combination of both walkers, or $X_{k}\left(t\right)\rightarrow Y=X_{j}+Z\left(X_{k}\left(t\right)-X_{j}\right)$.
* The new position is then accepted or rejected using the Metropolis-Hastings rule, as before.

The major feature of affine invariant ensemble samplers is that instead of using a single walker, a huge number of walkers can be deployed into the parameter space and all of the information across the ensemble is available to draw new proposals from. This uses the "parallel stretch move" which splits the ensemble into two sets and each set is used to update the other's position. Hence if some walkers detect an optimal probability, then more walkers can be brought to the area - and away from any problematic areas - to effectively sample the surrounding parameter space. The Markov quality is preserved however, as the proposal still only depends on the current state of the walkers and not the whole chain.

![Stretch Move](./assets/GoodmanWeare_StretchMove.png)

Goodman and Weare (2010) showed that this algorithm can increase sampling times by $>10$x compared to the Metropolis-Hastings algorithm. This is estimated by the autocorrelation time of the trace (value vs. model number) which is a measure of how often the sampler reaches a new area of parameter space.

In the cell below, three functions are defined that build the posterior probability distribution using the natural logarithm. This alters the accept/reject step from a ratio of probability values to a subtraction. I.e.:

* If $\ln fp\left(\theta^{\prime}|D\right)-\ln p\left(\theta_{k}|D\right)>\ln r$ then $\theta_{k+1}\leftarrow\theta^{\prime}$; otherwise $\theta_{k+1}\leftarrow\theta_{k}$.

This reduces the probability that the code with encounter underflow or overflow issues.

In [ ]:
def lnlike(q, data):
    """
Function to calculate the log-likelihood probability distribution of a model compared to data, using Gaussian statistics.
    """
    ll = []            # Empty list to store probabilities in
    x, y, yerr = data  # Separate data

    for m, c in q:      # Cycle through values
        ym = m * x + c  # Calculate the model
        ll.append(-0.5 * np.sum(((y - ym) / yerr) ** 2.))
    return np.array(ll)

def lnprior(q):
    """
Function to calculate the log-prior probability distribution. This is assumed to be a uniform distribution where the probability
is zero within the limits and negative-infinity otherwise.
    """
    u = np.array([0.5, 10.])  # Upper limits
    l = np.array([-5., 0.])   # Lower limits
    cond = (l[0]<q[:,0]) & (q[:,0]<u[0]) & (l[1]<q[:,1]) & (q[:,1]<u[1])
    # Return an array where 0. represents values within the limits, and negative infinity for those outside.
    lp = np.where(cond, 0., -np.inf)
    return lp

def lnprob(q, data):
    """
Function to calculate the posterior probability density function.
    """
    # Calculate the prior
    lp = lnprior(q)
    # Calculate the likelihood
    ll = lnlike(q, data)
    # Return the posterior
    lnp = np.where(lp == 0., lp + ll, -np.inf)
    return lnp

In [ ]:
def stretch_move(s, c, p0, Npars, data, a=2.):
    """
Function to perform the 'stretch move'.

    s : array. The sample to be updated.
    c : array. The complementary sample used to calculate new positions.
   p0 : array. The probabilities of s.
Npars : int. Number of free parameters.
 data : array. Required to calculate probabilities.
    a : float. The stretch parameter, can be fine-tuned to change the size of the steps taken.
    """
    Ns = len(s)
    Nc = len(c)
    
    zz = (((a - 1.) * np.random.rand(Ns) + 1.) ** 2.) / a  # Calculates random gradient for the affine linear transformation 
    rint = np.random.randint(Nc, size=(Ns,))  # Calculates a list of integers that will shuffle the complementary sample
    
    q = c[rint] - zz[:, np.newaxis] * (c[rint] - s)  # Propose new positions from the linear transformation
    p = lnprob(q, data)                              # Calculate the posterior probability of the new position
    
    p_diff = (Npars - 1) * np.log(zz) + p - p0             # Calucate the difference between the probabilities
    accept = p_diff > np.log(np.random.rand(len(p_diff)))  # Determine if the new positions are accepted
    
    return q, p, accept

In [ ]:
def proposal(q, p, data, Nwalk, Npars):
    """
Function to split the current sample into two sets. Then perform the stretch move on each sample to calculate new proposals and
add accepted values to the chain.

    q : array. Current positions.
    p : array. Current probabilities.
 data : array. Required to calculate new probabilities.
Nwalk : int. Number of walkers.
Npars : int. Number of free parameters.
    """
    half = int(Nwalk / 2)
    first, second = slice(half), slice(half, Nwalk)  # Functions to slice the arrays into two sets
    
    for S0, S1 in [(first, second), (second, first)]:                       # Loops over each set so they both are updated
        q_new, p_new, acc = stretch_move(q[S0], q[S1], p[S0], Npars, data)  # Use stretch move to calculate the proposal
        
        if np.any(acc):  # Add accepted values into the chains
            p[S0][acc] = p_new[acc]
            q[S0][acc] = q_new[acc]
    
    return q, p

In [ ]:
def Ensemble(q0, p0, data, Npars, Nwalk, Nstep):
    """
Function to iterate over the Markov chain ensemble.

   q0 : array. Initial positions.
   p0 : array. Initial probabilities.
 data : array. Data we're trying to find a p.d.f. for.
Npars : int. Number of free parameters.
Nwalk : int. Number of walkers.
Nstep : Number of Markov steps to take.
    """
    # Initialise arrays for the Markov chains and the probabilities. Set first elements to provided values.
    samples = np.ndarray((Nwalk, Nstep, Npars))
    samples[:,0,:] = np.array(q0)
    
    lnprob = np.ndarray((Nwalk, Nstep))
    lnprob[:,0] = np.array(p0)
    
    for i in range(1, Nstep):  # Iterate over the Markov steps
        q, p = proposal(samples[:,i-1,:], lnprob[:,i-1], data, Nwalk, Npars)  # Calculate and accept/reject the proposal
        
        # Add new positions to the chains.
        samples[:,i,:] = np.array(q)
        lnprob[:,i] = np.array(p)
    
    return samples, lnprob

In [ ]:
%%time
# True parameters
mt = -3.4975
ct = 2.479
true = [mt, ct]
name = ['m', 'c']

# Generate some test data
N = 50
x = np.sort(10 * np.random.rand(N))
y = mt * x + ct
yerr = 5. * np.random.rand(N)
y += np.random.normal(loc=0, scale=yerr)
data = np.array([x, y, yerr])

# Set up MCMC parameters
Npars = len(true)
Nwalk = 100
Nstep = 10000
Nburn = 100

# Calculate initial positions and probabilities
q0 = np.array([[-2.9, 1.9] + 1.e-4*np.random.randn(Npars) for i in range(Nwalk)])  # Gaussian ball centred on initial guess
p0 = lnprob(q0, data)

print("Total sample number =", Nwalk*Nstep)

# Run the ensemble sampler
chain = Ensemble(q0, p0, data, Npars, Nwalk, Nstep)
%timeit -n1 chain

samples, lnp = chain

In [ ]:
# Plot some figures
# Trace plots
fig, axes = plt.subplots(Npars, 1, sharex=True)

for i in range(Npars):
    for j in range(Nwalk):
        axes[i].plot(range(Nstep), samples[j,:,i], c='0.5', ls='-')
    axes[i].axvline(Nburn, c='r')
    axes[i].axhline(true[i], c='b')
    axes[i].set_ylabel(name[i])
axes[-1].set_xlabel('Model number')
fig.tight_layout()
plt.show()

# Corner plot
tmp = samples[:,Nburn:,:]
s = tmp.reshape(tmp.shape[0]*tmp.shape[1], tmp.shape[2])

fig, axes = plt.subplots(Npars, Npars)
fig.delaxes(axes[0,1])

# Histograms
for i in range(Npars):
    ax = axes[i,i]
    ax.hist(s[:,i], histtype='step', color='k', density=True)
    ax.axvline(true[i], c='b')
    if i == (Npars - 1):
        ax.set_yticks([])
        ax.set_xlabel(name[i])
    else:
        ax.set_xticks([])
        ax.set_yticks([])

# Densities
for i in range(Npars):
    for j in range(Npars):
        if i <= j:
            pass
        else:
            ax = axes[i,j]
            ax.plot(s[:,j], s[:,i], marker='.', c='0.5', ls=' ')
            ax.axvline(true[j], c='b')
            ax.axhline(true[i], c='b')
            if i < (Npars - 1):
                ax.set_xticks([])
            if (i == 2) & (j < Npars):
                ax.set_yticks([])
            if i == (Npars - 1):
                ax.set_xlabel(name[j])
            if j == 0:
                ax.set_ylabel(name[i])
fig.tight_layout()
plt.show()

# Plot data and model
m, c = np.mean(s[:,0]), np.mean(s[:,1])

print('m = {0:.3f} +/- {1:.3f} (true: {2:.3f})'.format(m, np.std(s[:,0]), mt))
print('c = {0:.3f} +/- {1:.3f} (true: {2:.3f})'.format(c, np.std(s[:,1]), ct))

plt.errorbar(x, y, yerr=yerr, fmt='.k', capsize=0)
plt.plot(x, mt*x+ct, '-b', label='True values')
plt.plot(x, m*x+c, '-r', label='Model')
plt.legend(loc='upper right')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Conclusion

MCMC is an important class of algorithm in science because it allows us to sample difficult pdfs of complex models without having a full analytical description, giving us an alternative method when other analytical approaches fall short. They are simple algorithms to construct, while being flexible and scalable to a variety of different scenarios, and can save valuable computational time by sampling only the most likely probabilities. The Affine Invariant Ensemble Sampler in particular is adept at navigating complex pdfs, compared to the M-H step, due to its ability to use the information gathered across the whole ensemble of walkers to draw proposals and step through parameter space. This knowledge of the other walkers' positions allows the ensemble to draw more walkers into regions of optimal probability and spend less time stuck in undesirable regions.

MCMC is a very powerful and popular tool within Astronomy and has successfully been implemented to [infer the orbit of a comet](https://arxiv.org/abs/1103.6038) and [fit the stellar structure of the Milky Way](https://arxiv.org/abs/1111.1724).

## References

* Foreman-Mackey, Hogg, Lang & Goodman (2013) Publications of the Astronomical Society of the Pacific, 125(925), 306
* Goodman & Weare (2010) Comm. App. Math & Comp. Sci, Vol. 5, No. 1
* Hogg & Foreman-Mackey (2017) ApJ Supplement series, 236(1), 11

* [Astrobites - Code you can use: the MCMC Hammer](https://astrobites.org/2012/02/20/code-you-can-use-the-mcmc-hammer/)
* [Augustinus Kristiadi - Tech Blog: Metropolis-Hastings](https://wiseodd.github.io/techblog/2015/10/17/metropolis-hastings/)
* [emcee: The MCMC Hammer Documentation](http://dfm.io/emcee/current/)